# Assignment 2 for Course 1MS041
Make         sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline         and your highest score will be used.

---
## Assignment 2, PROBLEM 1
Maximum Points = 8


## Random variable generation and transformation

The purpose of this problem is to show that you can implement your own sampler, this will be built in the following three steps:

1. [2p] Implement a Linear Congruential Generator where you tested out a good combination (a large $M$ with $a,b$ satisfying the Hull-Dobell (Thm 6.8)) of parameters. Follow the instructions in the code block.
2. [2p] Using a generator construct random numbers from the uniform $[0,1]$ distribution.
3. [4p] Using a uniform $[0,1]$ random generator, generate samples from 

$$p_0(x) = \frac{\pi}{2}|\sin(2\pi x)|, \quad x \in [0,1] \enspace .$$

Using the **Accept-Reject** sampler (**Algorithm 1** in TFDS notes) with sampling density given by the uniform $[0,1]$ distribution.

In [2]:

def problem1_LCG(size=None, seed = 0):
    """
    A linear congruential generator that generates pseudo random numbers according to size.
    
    Parameters
    -------------
    size : an integer denoting how many samples should be produced
    seed : the starting point of the LCG, i.e. u0 in the notes.
    
    Returns
    -------------
    out : a list of the pseudo random numbers
    """
    
    M = 2**31
    a = 5**15
    b = 1
    x = seed # the seed
    retValue = [x % M]  # start the list with x=x0
    for i in range(2, size+1, 1):
        x = (a * x + b) % M # the generator, using modular arithmetic
        retValue.append(x) # append the new x to the list
    return retValue

In [3]:

def problem1_uniform(generator=None, period = 1, size=None, seed=0):
    """
    Takes a generator and produces samples from the uniform [0,1] distribution according
    to size.
    
    Parameters
    -------------
    generator : a function of type generator(size,seed) and produces the same result as problem1_LCG, i.e. pseudo random numbers in the range {0,1,...,period-1}
    period : the period of the generator
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the uniform pseudo random numbers
    """
    
    a = [(x%period)/period for x in generator(size, seed)]
    return a


In [4]:
import math 
import numpy as np


def f1(x):
    return (np.pi/2)*abs(math.sin(x*2*np.pi))

def problem1_accept_reject(uniformGenerator=None, n_iterations=None, seed=0):
    """
    Takes a generator that produces uniform pseudo random [0,1] numbers 
    and produces samples from (pi/2)*abs(sin(x*2*pi)) using an Accept-Reject
    sampler with the uniform distribution as the proposal distribution
    
    Parameters
    -------------
    generator : a function of the type generator(size,seed) that produces uniform pseudo random
    numbers from [0,1]
    seed : the seed to be used in the generator provided
    size : an integer denoting how many samples should be produced
    
    Returns
    --------------
    out : a list of the pseudo random numbers with the specified distribution
    """
    c = 2
    i = 0
    pos = 0
    sample = uniformGenerator(30*n_iterations, seed)
    output = np.zeros(n_iterations)
    while i<n_iterations and pos < len(sample):
        x = sample[pos]
        u = sample[pos+1]
        r = f1(x)/(c)
        if u <= r:
            output[i] = x
            i = i + 1 
        pos = pos +2
    return output

---
#### Local Test for Assignment 2, PROBLEM 1
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [5]:

# If you managed to solve all three parts you can test the following code to see if it runs
# you have to change the period to match your LCG though, this is marked as XXX.
# It is a very good idea to check these things using the histogram function in sagemath
# try with a larger number of samples, up to 10000 should run

print("LCG output: %s" % problem1_LCG(size=10, seed = 1))

period = 2147483648

print("Uniform sampler %s" % problem1_uniform(generator=problem1_LCG, period = period, size=10, seed=1))

uniform_sampler = lambda size,seed: problem1_uniform(generator=problem1_LCG, period = period, size=size, seed=seed)

print("Accept-Reject sampler %s" % problem1_accept_reject(uniformGenerator = uniform_sampler,n_iterations=20,seed=1))

LCG output: [1, 452807054, 886112567, 2043763276, 1076520925, 107100602, 2002900339, 1197955928, 49375097, 34373286]
Uniform sampler [4.656612873077393e-10, 0.210854715667665, 0.41262831864878535, 0.9517014380544424, 0.5012941197492182, 0.049872604198753834, 0.9326731502078474, 0.5578416995704174, 0.022992071229964495, 0.01600630860775709]
Accept-Reject sampler [0.02299207 0.27811789 0.24337863 0.48040049 0.83002768 0.67560932
 0.27638148 0.22475216 0.67771475 0.28882483 0.8420597  0.36976025
 0.32446474 0.70388313 0.38701159 0.71252057 0.33013808 0.66140409
 0.3171319  0.12982931]


In [6]:

# If however you did not manage to implement either part 1 or part 2 but still want to check part 3, you can run the code below

def testUniformGenerator(size,seed):
    import random
    random.seed(seed)
    
    return [random.uniform(0,1) for s in range(size)]

print("Accept-Reject sampler %s" % problem1_accept_reject(uniformGenerator=testUniformGenerator, n_iterations=20, seed=1))

Accept-Reject sampler [0.76377462 0.09385959 0.8357651  0.76228008 0.90142746 0.2165994
 0.23308445 0.21878104 0.28978161 0.83757798 0.64229436 0.12088996
 0.72148441 0.83003569 0.30336851 0.79740425 0.1730074  0.70304076
 0.37470302 0.48969352]


---
## Assignment 2, PROBLEM 2
Maximum Points = 8


## Markovian travel

The dataset `Travel Dataset - Datathon 2019` is a simulated dataset designed to mimic real corporate travel systems -- focusing on flights and hotels. The file is at `data/flights.csv` in the same folder as `Exam.ipynb`, i.e. you can use the path `data/flights.csv` from the notebook to access the file.

1. [2p] In the first code-box 
    1. Load the csv from file `data/flights.csv`
    2. Fill in the value of the variables as specified by their names.
2. [2p] In the second code-box your goal is to estimate a Markov chain transition matrix for the travels of these users. For example, if we enumerate the cities according to alphabetical order, the first city `'Aracaju (SE)'` would correspond to $0$. Each row of the file corresponds to one flight, i.e. it has a starting city and an ending city. We model this as a stationary Markov chain, i.e. each user's travel trajectory is a realization of the Markov chain, $X_t$. Here, $X_t$ is the current city the user is at, at step $t$, and $X_{t+1}$ is the city the user travels to at the next time step. This means that to each row in the file there is a corresponding pair $(X_{t},X_{t+1})$. The stationarity assumption gives that for all $t$ there is a transition density $p$ such that $P(X_{t+1} = y | X_t = x) = p(x,y)$ (for all $x,y$). The transition matrix should be `n_cities` x `n_citites` in size.
3. [2p] Use the transition matrix to compute out the stationary distribution.
4. [2p] Given that we start in 'Aracaju (SE)' what is the probability that after 3 steps we will be back in 'Aracaju (SE)'?

In [9]:
import numpy as np
import csv

filename = open('data/flights.csv', 'r')
file =csv.DictReader(filename)

cities = []
userCodes = []
observations = []
to = []

for col in file:
    cities.append(col['from'])
    userCodes.append(col['userCode'])
    observations.append(col['travelCode'])
    to.append(col['to'])

number_of_cities = len(set(cities))
number_of_userCodes = len(set(userCodes))
number_of_observations = len((observations))

In [10]:

# This is a very useful function that you can use for part 2. You have seen this before when parsing the
# pride and prejudice book.

def makeFreqDict(myDataList):
    '''Make a frequency mapping out of a list of data.

    Param myDataList, a list of data.
    Return a dictionary mapping each unique data value to its frequency count.'''

    freqDict = {} # start with an empty dictionary

    for res in myDataList:
        if res in freqDict: # the data value already exists as a key
                freqDict[res] = freqDict[res] + 1 # add 1 to the count using sage integers
        else: # the data value does not exist as a key value
            freqDict[res] = 1 # add a new key-value pair for this new data value, frequency 1

    return freqDict # return the dictionary created

In [11]:
from collections import Counter

unique_cities = sorted(set(cities))
n_cities = len(unique_cities) 
transitions = list(zip(cities, to))
transition_counts = (makeFreqDict(transitions))
indexToCity = {0:'Aracaju (SE)', 1:'Brasilia (DF)', 2:'Campo Grande (MS)', 3:'Florianopolis (SC)', 
               4:'Natal (RN)', 5:'Recife (PE)',6: 'Rio de Janeiro (RJ)', 7:'Salvador (BH)', 8:'Sao Paulo (SP)'}

cityToIndex = {'Aracaju (SE)': 0, 'Brasilia (DF)': 1, 'Campo Grande (MS)': 2, 'Florianopolis (SC)':3, 
               'Natal (RN)': 4, 'Recife (PE)':5, 'Rio de Janeiro (RJ)':6, 'Salvador (BH)':7 , 'Sao Paulo (SP)': 8}

# print(transition_counts)
# transition_matrix = XXX # a numpy array of size (n_cities,n_cities)
transition_matrix = np.zeros([len(unique_cities),len(unique_cities)])
for c1 in unique_cities:
    for c2 in unique_cities:
        try:
            transition_matrix[cityToIndex[c1]][cityToIndex[c2]]=transition_counts[(c1,c2)]
        except:
            transition_matrix[cityToIndex[c1]][cityToIndex[c2]]=0


b = np.sum(transition_matrix, axis=1)

transition_matrix = transition_matrix / b
transition_matrix


# The transition matrix should be ordered in such a way that
# p_{'Aracaju (SE)','Rio de Janeiro (RJ)'} = transition_matrix[cityToIndex['Aracaju (SE)'],cityToIndex['Rio de Janeiro (RJ)']]
# and represents the probability of travelling Aracaju (SE)->Rio de Janeiro (RJ)

# Make sure that the transition_matrix does not contain np.nan from division by zero for instance

array([[0.        , 0.15702265, 0.15520318, 0.15079296, 0.16544797,
        0.16023622, 0.16758846, 0.17060337, 0.1607619 ],
       [0.12983559, 0.        , 0.12999309, 0.1357189 , 0.1255253 ,
        0.1253937 , 0.1185846 , 0.1174579 , 0.12012698],
       [0.14487965, 0.14675591, 0.        , 0.14398869, 0.14889057,
        0.14796588, 0.14362177, 0.14733396, 0.15225397],
       [0.23218891, 0.25273726, 0.23751007, 0.        , 0.28193814,
        0.24963911, 0.34534642, 0.33910196, 0.28431746],
       [0.1057651 , 0.09704669, 0.1019627 , 0.11705079, 0.        ,
        0.09494751, 0.05649718, 0.05413938, 0.07830688],
       [0.13120567, 0.12417557, 0.12979164, 0.13275294, 0.12161708,
        0.        , 0.11281594, 0.11412535, 0.12325926],
       [0.07570385, 0.06478443, 0.0695004 , 0.10131375, 0.03992268,
        0.06223753, 0.        , 0.        , 0.03953439],
       [0.07839029, 0.06527178, 0.07252216, 0.10119162, 0.0389141 ,
        0.06404199, 0.        , 0.        , 0.04143915],


In [12]:


# This should be a numpy array of length n_cities which sums to 1 and is all positive

evals,evecs = np.linalg.eig(transition_matrix)
evals,evecs
first_evec = evecs[:,0]
stationary_distribution_problem2 = np.array(first_evec) / np.sum(first_evec)
stationary_distribution_problem2

array([0.13690932, 0.1132047 , 0.12780262, 0.21081107, 0.08752133,
       0.11210498, 0.06184532, 0.06290826, 0.0868924 ])

In [17]:

# Compute the return probability for part 3 of PROBLEM 2
p = transition_matrix
p3 = p@p@p
print(p3)
return_probability_problem2 = p3[0][0]

[[0.13331718 0.13751552 0.13718491 0.13597117 0.13839326 0.13771913
  0.13891645 0.13897523 0.13818637]
 [0.11370595 0.11120097 0.1136094  0.11479672 0.11276454 0.11317307
  0.11213056 0.112132   0.11259308]
 [0.12805988 0.12825951 0.12503851 0.12782311 0.128441   0.1283191
  0.12841955 0.12849316 0.1285695 ]
 [0.20936653 0.21377574 0.21084486 0.19215054 0.21959695 0.21361966
  0.22651528 0.22613885 0.21970023]
 [0.08846997 0.08718103 0.0879585  0.09116892 0.08491183 0.08708216
  0.08385637 0.08388434 0.08541416]
 [0.11276808 0.11207366 0.11255802 0.11359854 0.11154245 0.11014244
  0.11091312 0.11094356 0.11159889]
 [0.06275199 0.0612585  0.06214386 0.06645244 0.05925554 0.0611878
  0.05745764 0.05752516 0.05927222]
 [0.06385752 0.06231215 0.06324816 0.06748223 0.06029407 0.06225652
  0.05851384 0.05857762 0.06032921]
 [0.0877029  0.08642293 0.08741379 0.09055634 0.08480036 0.08650012
  0.0832772  0.08333007 0.08433636]]


---
#### Local Test for Assignment 2, PROBLEM 2
Evaluate cell below to make sure your answer is valid.                             You **should not** modify anything in the cell below when evaluating it to do a local test of                             your solution.
You may need to include and evaluate code snippets from lecture notebooks in cells above to make the local test work correctly sometimes (see error messages for clues). This is meant to help you become efficient at recalling materials covered in lectures that relate to this problem. Such local tests will generally not be available in the exam.

In [18]:
# Once you have created all your functions, you can make a small test here to see
# what would be generated from your model.
import numpy as np

start = np.zeros(shape=(n_cities,1))
start[cityToIndex['Aracaju (SE)'],0] = 1

current_pos = start
for i in range(10):
    random_word_index = np.random.choice(range(n_cities),p=current_pos.reshape(-1))
    current_pos = np.zeros_like(start)
    current_pos[random_word_index] = 1
    print(indexToCity[random_word_index],end='->')
    current_pos = (current_pos.T@transition_matrix).T

Aracaju (SE)->

ValueError: probabilities do not sum to 1

---
## Assignment 2, PROBLEM 3
Maximum Points = 4



Derive the maximum likelihood estimate for $n$ IID samples from a random variable with the following probability density function:
$$
f(x; \lambda) = \frac{1}{24} \lambda^5 x^4 \exp(-\lambda x), \qquad \text{ where, } \lambda>0, x > 0
$$

You can solve the MLe by hand (using pencil paper or using key-strokes). Present your solution as the return value of a function called `def MLeForAssignment2Problem3(x)`, where `x` is a list of $n$ input data points.

In [15]:

# do not change the name of the function, just replace XXX with the appropriate expressions for the MLe
def MLeForAssignment2Problem3(x):
    '''write comment of what this function does'''
    tn=sum(x)
    n=len(x)
    return (5*n)/tn

---
## Assignment 2, PROBLEM 4
Maximum Points = 4


Use the **Multi-dimensional Constrained Optimisation** example (in `07-Optimization.ipynb`) to numerically find the MLe for the mean and variance parameter based on `normallySimulatedDataSamples`, an array obtained by a specific simulation of $30$ IID samples from the $Normal(10,2)$ random variable.

Recall that $Normal(\mu, \sigma^2)$ RV has the probability density function given by:

$$
f(x ;\mu, \sigma) = \displaystyle\frac{1}{\sigma\sqrt{2\pi}}\exp\left(\frac{-1}{2\sigma^2}(x-\mu)^2\right)
$$

The two parameters, $\mu \in \mathbb{R} := (-\infty,\infty)$ and $\sigma \in (0,\infty)$, are sometimes referred to as the location and scale parameters.

You know that the log likelihood function for $n$ IID samples from a Normal RV with parameters $\mu$ and $\sigma$ simply follows from $\sum_{i=1}^n \log(f(x_i; \mu,\sigma))$, based on the IID assumption. 

NOTE: When setting bounding boxes for $\mu$ and $\sigma$ try to start with some guesses like $[-20,20]$ and $[0.1,5.0]$ and make it larger if the solution is at the boundary. Making the left bounding-point for $\sigma$ too close to $0.0$ will cause division by zero Warnings. Other numerical instabilities can happen in such iterative numerical solutions to the MLe. You need to be patient and learn by trial-and-error. You will see the mathematical theory in more details in a future course in scientific computing/optimisation. So don't worry too much now except learning to use it for our problems.  

In [16]:


import numpy as np
from scipy import optimize
from math import pi
# do NOT change the next three lines
np.random.seed(123456) # set seed
# simulate 30 IID samples drawn from Normal(10,2)RV
normallySimulatedDataSamples = np.random.normal(10,2,30) 

# define the negative log likelihoo function you want to minimise by editing XXX
def negLogLklOfIIDNormalSamples(parameters):
    '''return the -log(likelihood) of normallySimulatedDataSamples with mean and var parameters'''
    mu_param=parameters[0]
    sigma_param=parameters[1]
    return -np.sum( np.log( (1/(sigma_param*np.sqrt(2*pi)))*np.exp((-((normallySimulatedDataSamples-mu_param)**2))/(2*(sigma_param**2)))))


# you should only change XXX below and not anything else
parameter_bounding_box=((-20, 20), (.3, 5)) # specify the constraints for each parameter - some guess work...
initial_arguments = np.array([0, 1]) # point in 2D to initialise the minimize algorithm
result_Ass2Prob4 = optimize.minimize(negLogLklOfIIDNormalSamples, initial_arguments, bounds=parameter_bounding_box) 
# call the minimize method above finally! you need to play a bit to get initial conditions and bounding box ok
result_Ass2Prob4


      fun: 58.631387282367946
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-7.10542677e-07,  6.39488466e-06])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 66
      nit: 11
     njev: 22
   status: 0
  success: True
        x: array([9.26861949, 1.70820174])